<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/PFR_Exothermic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.integrate import solve_ivp
import numpy as np

from plotly.subplots import make_subplots

In [22]:
N=25
q = 100 #L/min
c_ai = 1 #mol/L
Ti = 350 #K
total_V = 100 #L
V = total_V/N  #L discretized volume
rho = 1000 #g/L
C = 0.239 #J/g K
neg_Hr = 5e4 #J/mol
ER = 8750 #K
k0 = 7.2e10 #/min
total_UA = 5e4 #J/min K
UA = total_UA/N  #discretized overall heat transfer coefficient*area
Tc0 = 300. #K
c_a0 = 0.5#mol/L
T0 = 350 #K

In [23]:
def rhs(t,vec):
    T, c = np.split(vec,2)

    k = k0*np.exp(-ER/T)
    c_ins = np.r_[c_ai, c[:-1]]
    dca = q/V*(c_ins - c) - k*c

    Tins = np.r_[Ti, T[:-1]]
    dT = q/V*(Tins - T) + k*c * (neg_Hr)/(rho*C) - UA* (T - 305.) / (rho * V * C)
    return np.r_[dT,dca]



In [26]:
tend = 1. #min
res = solve_ivp(rhs, (0,tend), np.r_[np.repeat(T0,N),np.repeat(c_a0,N)], method="Radau", dense_output=True)

tplot = np.linspace(0,tend,10)
T_Cs = res.sol(tplot).T


In [27]:
fig = make_subplots(1,2)

pos = np.arange(N)
for T_C in T_Cs:
    T, C = np.split(T_C,2)
    fig.add_scatter(x=pos, y=T, row=1, col=1,)
    fig.add_scatter(x=pos, y=C, row=1, col=2,)
fig.update_layout(height=400,width=800, template='plotly_dark')